In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 30
pd.options.display.max_columns = 100

In [ ]:
rawdata = pd.read_csv('fe/rawsim2.csv',sep=',',decimal='.',encoding='ANSI')

slicemeta = pd.read_csv('fe/slicemeta2.csv',sep=',',decimal='.',encoding='ANSI')

metadata = pd.read_csv('fe/aggregations_metadata4.csv',sep=',',decimal='.',encoding='ANSI')
agglist = pd.read_csv('fe/aggregations_agglist3.csv',sep=',',decimal='.',encoding='ANSI')
varcomb = pd.read_csv('fe/varcomb.csv',sep=',',decimal='.',encoding='ANSI')

metadataR = pd.read_csv('fe/aggregations_metadataR.csv',sep=',',decimal='.',encoding='ANSI')
agglistR = pd.read_csv('fe/aggregations_agglistR.csv',sep=',',decimal='.',encoding='ANSI')
seglistR = pd.read_csv('fe/aggregations_segmlist.csv',sep=',',decimal='.',encoding='ANSI')

timesincemeta = pd.read_csv('fe/timesincemeta2.csv',sep=',',decimal='.',encoding='ANSI')

issomemeta = pd.read_csv('fe/issomemeta.csv',sep=',',decimal='.',encoding='ANSI')

intermeta = pd.read_csv('fe/intermeta.csv',sep=',',decimal='.',encoding='ANSI')

catmeta = pd.read_csv('fe/catmeta.csv',sep=',',decimal='.',encoding='ANSI')
catmetab = pd.read_csv('fe/catmetab.csv',sep=',',decimal='.',encoding='ANSI')

In [ ]:
import sys
sys.path.insert(0, 'C:/py_src/scoring/hcfsc')
import scoring
import scoring.feature_engineering

In [ ]:
### RELOADING MODULE FOR TESTING PURPOSES

import importlib
importlib.reload(scoring)
importlib.reload(scoring.feature_engineering)

In [ ]:
display(rawdata)

In [ ]:
from scoring.feature_engineering import OrderAssigner

oa = OrderAssigner('TIME', 'TIME_APPLICATION', time_granularity = 'months', history_length = 6, time_format = '%Y-%m-%d %H:%M:%S')
oa.fit(rawdata)
orderdata = oa.transform(rawdata)

In [ ]:
print(oa.strsql_)
display(orderdata)

In [ ]:
from scoring.feature_engineering import FeatureEngineeringFromSlice

fe1 = FeatureEngineeringFromSlice('ID_APPLICATION', 'TIME_ORDER', metadata=metadataR, agglist=agglistR, segm=seglistR, max_time = 6)
fe1.fit(orderdata)
transrawdata = fe1.transform(orderdata)

In [ ]:
print(fe1.strsql_)
display(transrawdata)

In [ ]:
importlib.reload(scoring.feature_engineering)
from scoring.feature_engineering import Slicer

sl = Slicer('ID_APPLICATION', 'TIME', 'TIME_APPLICATION', slicemeta=slicemeta, time_granularity='months', history_length=6,  time_format = '%Y-%m-%d %H:%M:%S')
sl.fit(rawdata)
newdata = sl.transform(rawdata)

In [ ]:
print(sl.strsql_)
display(newdata)

In [ ]:
from scoring.feature_engineering import FeatureEngineeringFromSlice

fe2 = FeatureEngineeringFromSlice('ID_APPLICATION', 'TIME_ORDER', metadata=metadata, agglist=agglist, varcomb=varcomb, max_time = 6)
fe2.fit(newdata)
transdata = fe2.transform(newdata)

In [ ]:
print(fe2.strsql_)
display(transdata)

In [ ]:
timesincemeta

In [ ]:
from scoring.feature_engineering import TimeSinceCalc

tsc = TimeSinceCalc('ID_APPLICATION','TIME','TIME_APPLICATION',timesincemeta=timesincemeta, time_format='%Y-%m-%d %H:%M:%S', keyword='entry')
tsc.fit(rawdata)
ds = tsc.transform(rawdata)

In [ ]:
print(tsc.strsql_)
display(ds)

In [ ]:
from scoring.feature_engineering import IsSomething

iss = IsSomething('ID_APPLICATION','TIME_ORDER',issomemeta)
iss.fit(newdata)
issomedata = iss.transform(newdata)

In [ ]:
print(iss.strsql_)
display(issomedata)

In [ ]:
from scoring.feature_engineering import CategoricalFeatures

ct = CategoricalFeatures('ID_APPLICATION', 'raw', catmeta = catmeta, slice_name = 'TIME_ORDER', time_name = 'TIME',
                        time_max_name = 'TIME_APPLICATION')
ct.fit(orderdata)
catdata = ct.transform(orderdata)

In [ ]:
print(ct.strsql_)
display(catdata)

In [ ]:
from scoring.feature_engineering import CategoricalFeatures

ctb = CategoricalFeatures('ID_APPLICATION', 'slice', catmeta = catmetab, slice_name = 'TIME_ORDER')
ctb.fit(newdata)
catdatab = ctb.transform(newdata)

In [ ]:
print(ctb.strsql_)
display(catdatab)

In [ ]:
from scoring.feature_engineering import Interactions

itr = Interactions(intermeta)
itr.fit(rawdata)
itrdata = itr.transform(rawdata)

In [ ]:
print(itr.strsql_)
display(itrdata)